# Import data

In [1]:
import os
import json
from torchvision import datasets, transforms
from torch.utils.data import random_split, Subset

DATASET_PATH = 'dataset'
TEST_INDICES_PATH = 'test_indices.json'
transform = transforms.ToTensor()
dataset = datasets.ImageFolder(root=DATASET_PATH, transform=transform)
total_size = len(dataset)

train_size = int(0.8 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size

if not os.path.exists(TEST_INDICES_PATH):
    # Create test split for the first time
    indices = list(range(total_size))
    splits = random_split(indices, [train_size, val_size, test_size])
    train_indices, val_indices, test_indices = splits[0].indices, splits[1].indices, splits[2].indices
    with open(TEST_INDICES_PATH, 'w') as f:
        json.dump(test_indices, f)
else:
    # Load existing test split
    with open(TEST_INDICES_PATH, 'r') as f:
        test_indices = json.load(f)
    # Use the rest for train/val; here is one way to handle it:
    train_val_indices = list(set(range(total_size)) - set(test_indices))
    temp_dataset = Subset(dataset, train_val_indices)
    train_val_size = len(temp_dataset)
    train_size = int(0.8 * train_val_size)
    val_size = train_val_size - train_size
    splits = random_split(train_val_indices, [train_size, val_size])
    train_indices, val_indices = splits[0].indices, splits[1].indices

train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)
test_dataset = Subset(dataset, test_indices)


# Data Loader

In [2]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


# Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SignNet(nn.Module):
    def __init__(self, num_classes):
        super(SignNet, self).__init__()
        # Block 1
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm2d(32)

        # Block 2
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm2d(64)

        # Block 3
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        # Block 4
        self.pool1 = nn.MaxPool2d(2)

        # Block 5
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool2 = nn.MaxPool2d(2)

        # Block 6
        self.conv5 = nn.Conv2d(256, 384, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(384)
        self.pool3 = nn.MaxPool2d(2)

        # Block 7
        self.conv6 = nn.Conv2d(384, 512, kernel_size=3, padding=1)
        self.bn6 = nn.BatchNorm2d(512)

        # GAP Layer
        self.gap = nn.AdaptiveAvgPool2d((1, 1))

        # FC Layers
        self.fc1 = nn.Linear(512, 84)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc_out = nn.Linear(84, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # Block 1
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)

        # Block 2
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)

        # Block 3
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)

        # Block 4
        x = self.pool1(x)

        # Block 5
        x = self.conv4(x)
        x = self.bn4(x)
        x = F.relu(x)
        x = self.pool2(x)

        # Block 6
        x = self.conv5(x)
        x = self.bn5(x)
        x = F.relu(x)
        x = self.pool3(x)

        # Block 7
        x = self.conv6(x)
        x = self.bn6(x)
        x = F.relu(x)

        # Global Average Pooling
        x = self.gap(x)
        x = x.view(x.size(0), -1)

        # Fully Connected
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc_out(x)
        x = self.softmax(x)
        return x


In [ ]:
model = SignNet(num_classes=25)


